In [39]:
from google.colab import data_table
# uncomment the following line to see all table
data_table.enable_dataframe_formatter()

# comment the following line to disable this feature
#data_table.disable_dataframe_formatter()

# **Task1**

In [40]:
import pandas

# uncomment the following codes if need to restart
#import shutil
#shutil. rmtree("EECS4312")

# get "MyAppDescriptions.csv" and "MyAppFeatures.csv" from git
!git clone https://github.com/li-jiahao-216263949/EECS4312.git

fatal: destination path 'EECS4312' already exists and is not an empty directory.


In [41]:
# read csv files
features = pandas.read_csv('EECS4312/MyAppFeatures.csv')
descriptions = pandas.read_csv('EECS4312/MyAppDescriptions.csv')
lists = pandas.read_csv('EECS4312/MyAppLists.csv')
bigrams = pandas.read_csv('EECS4312/BiGrams.csv')
trigrams = pandas.read_csv('EECS4312/TriGrams.csv')

In [42]:
# read the 5 csv into memory
reviews1 = pandas.read_csv('EECS4312/'+ lists['package name'].iloc[0]+ '.csv')
reviews2 = pandas.read_csv('EECS4312/'+ lists['package name'].iloc[5]+ '.csv')
reviews3 = pandas.read_csv('EECS4312/'+ lists['package name'].iloc[10]+ '.csv')
reviews4 = pandas.read_csv('EECS4312/'+ lists['package name'].iloc[15]+ '.csv')
reviews5 = pandas.read_csv('EECS4312/'+ lists['package name'].iloc[20]+ '.csv')

In [43]:
# cat all reviews in a single dataframe
reviews = [reviews1,reviews2,reviews3,reviews4,reviews5]
reviews = pandas.concat(reviews)

# **Task2**<br>

**(i) The code snippet**

In [44]:
!pip install emoji --upgrade
import emoji
import re
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
nltk.download('wordnet')

# remove punction in description
# there is no ' in descriptions, so we do not need to consider the tokenization
def remove_punction(text):
  return "".join([char for char in str(text) if char not in string.punctuation])

remove_special = lambda x : re.sub(r"[^a-zA-Z0-9]+", ' ', x)
# remove emoji
def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

nums = {'0':'zero', '1':'one', '2':'two', '3':'three', '4':"four", '5':"five", '6':"six", '7':"seven", '8':"eight", '9':"nine"}
# check all numbers, replace it with word if any
def turn_num2word(text):
  s = text
  for n in nums:
    if n in s:
      s = s.replace(n, nums[n])
  return s

# split the string and join them with only 1 space
leave_one_space = lambda x : " ".join(x.split())

# convert it to lowercase
to_lowercase = lambda x : x.lower()

# get stop words from nltk
stop_words = stopwords.words('english')

def remove_stopword(text):
  return " ".join([word for word in text.split() if word not in stop_words])

lemma = nltk.stem.WordNetLemmatizer()
def Lemmatization(text):
  return " ".join([lemma.lemmatize(word) for word in text.split()])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
# apply the functions for all reviews
reviews["Review"] = reviews["Review"].apply(remove_punction)
reviews["Review"] = reviews["Review"].apply(remove_special)
reviews["Review"] = reviews["Review"].apply(remove_emoji)
reviews["Review"] = reviews["Review"].apply(turn_num2word)
reviews["Review"] = reviews["Review"].apply(leave_one_space)
reviews["Review"] = reviews["Review"].apply(to_lowercase)
reviews["Review"] = reviews["Review"].apply(remove_stopword)
reviews["Review"] = reviews["Review"].apply(Lemmatization)

**(ii) sample output of 15 Pre-processed reviews**

In [46]:
sample_reviews = reviews.sample(15)
for i in range(15):
  print(sample_reviews["Review"].iloc[i])

nice app
love paid version many great resource love daily information reason gave four star got rid haircut calendar depended reason im growing hair one main reason went app
absolutely adore ap
love game
excellent using year
game played get make many move like made box make box make one line make another box get make one line guy dont even know play
nice financial app
excellent
good app loan calculation forget negative review complaining ad option spend money buy ad free version keep using im sure like
lovely
fantastic
awesome
great app never seen
best
excellent game


# **Bonues of Task2** <br>
Answer: <br>
Yes, the choice of stop words will impact on the analysis. <br>
Using non-customized list of stop words may cause the result of our analysis become meaningless. There are many words related to the specific app, but the appearence of those high frequence words are not helpful in analysis. <br>
For example, I have a app called "Loan calculator". This app is helping people to calculate the loan and its monthly payment. Obviously, in the review, the clients will of course using a lot of "calculate". But this word is not helping in the elicitation for the future analysis.


# **Task3**

**(I) Use Textblob for calculating Reviews Sentiment (code snippet)**

In [47]:
from textblob import TextBlob

In [48]:
# define the method to return polarity by textblob
def get_polarity_textblob(review):
  temp = TextBlob(review)
  return temp.sentiment.polarity

In [49]:
# get package name and review from reviews table
polarity_table = reviews[['Package Name','Review']]
polarity_table.columns = ['App\'s Package Name','Review']
polarity_table.reset_index(drop=True, inplace=True)

In [50]:
tmp = pandas.DataFrame()
tmp['Polarity'] = pandas.DataFrame(polarity_table['Review'].apply(get_polarity_textblob))
TB_polarity = pandas.concat([polarity_table, tmp], axis=1)

**(II) display the table for Textblob**

In [51]:
# show only the first 250 reviews for the textblob
display(TB_polarity.head(250))

,App's Package Name,Review,Polarity
0,com.google.android.diskusage,absolutely love app design easy use,0.466667
1,com.google.android.diskusage,doesnt work anymore,0.000000
2,com.google.android.diskusage,good,0.700000
3,com.google.android.diskusage,great app exactly supposed,0.525000
4,com.google.android.diskusage,great app searching something similar windirst...,0.466667
...,...,...,...
245,com.google.android.diskusage,perfect sequoiaview enjoyer,1.000000
246,com.google.android.diskusage,would nice supported android onezero dark them...,0.216667
247,com.google.android.diskusage,brilliant app simple intuitive user interface ...,0.475000
248,com.google.android.diskusage,useful longer updated,0.300000


**(III) Use Vader for calculating Reviews Sentiment (code snippet)**

In [52]:
# install vader
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [53]:
analyzer = SentimentIntensityAnalyzer()
# def function return the polarity of reviews
def get_polarity_vader(review):
  return analyzer.polarity_scores(review)['compound']

In [54]:
tmp['Polarity'] = pandas.DataFrame(polarity_table['Review'].apply(get_polarity_vader))
Vader_polarity = pandas.concat([polarity_table, tmp], axis=1)

**(IV) display the table for Vader**

In [55]:
# show only the first 250 reviews for the vader
display(Vader_polarity.head(250))

,App's Package Name,Review,Polarity
0,com.google.android.diskusage,absolutely love app design easy use,0.8122
1,com.google.android.diskusage,doesnt work anymore,0.0000
2,com.google.android.diskusage,good,0.4404
3,com.google.android.diskusage,great app exactly supposed,0.6249
4,com.google.android.diskusage,great app searching something similar windirst...,0.8360
...,...,...,...
245,com.google.android.diskusage,perfect sequoiaview enjoyer,0.7845
246,com.google.android.diskusage,would nice supported android onezero dark them...,0.8369
247,com.google.android.diskusage,brilliant app simple intuitive user interface ...,0.8402
248,com.google.android.diskusage,useful longer updated,0.4404


**(V) The comparison**

In [56]:
TB_samples = TB_polarity.sample(10)
samples_index = TB_samples.index
Vader_samples = Vader_polarity.iloc[samples_index]
comparisons = pandas.concat([TB_samples, Vader_samples['Polarity']], axis=1)
cols = ["App's Package Name",'Review','TextBlob Polarity', 'Vader Polarity']
comparisons.columns = cols
comparisons

,App's Package Name,Review,TextBlob Polarity,Vader Polarity
7414,com.financial.calculator,great app,0.800000,0.6249
7612,com.financial.calculator,best financial calculator benefitted immensely,0.333333,0.7845
2110,com.mobile_infographics_tools.mydrive,,0.000000,0.0000
13679,com.moonly.android,amazing far,0.350000,0.5859
920,com.google.android.diskusage,amazing fast dont mind ui,0.400000,0.5859
14983,com.burakgon.dnschanger,nice,0.600000,0.4215
2863,com.mobile_infographics_tools.mydrive,good,0.700000,0.4404
14106,com.moonly.android,beautifully done much information realized love,0.516667,0.8360
12230,com.itbenefit.android.calendar,bad widget tbf could option sync calendar like...,-0.700000,-0.2500
14215,com.moonly.android,love learn go downloaded today,0.500000,0.6369


**Answer:**<br>
The Textblob and and Vader are very different in calculating the polarity.<br>
<br>
**Textblob:** <br>
This approach use the lexicon in "en-sentiment.xml". Each word in the lexicon may have scores, which contains the polarity and subjectivity. When there are multiple lexicons, it simply calculate the average for calculation. However, TextBlob doesn't not handle negation in the way it calculating the positive words. It apply a negative multiplier on the polarity. As for the neutral words, it doesn't care.
<br><br>
**Vader:**<br>
This will return a dictionary which give the score of positive, neutral, negative and compound. The compound is the polarity we need. The compound score is the sum of positive, negative & neutral scores which is then normalized between -1(most extreme negative) and +1 (most extreme positive). <br>
<br>
**Conclusion:**
Vader is better in the review analysis.<br>
**Reason:** <br>
Textblob works strongly on the formal language usage, because the lexicons in the "en-sentiment.xml" are more close to the words in dictionary. However the reviews are more casually on the google play. Vader works better with slang, emojis. So vader is better way to do the review analysis.

# **Task4**

**I. Among your five ASSIGNED APPs, choose the one with the highest number of
reviews.**

In [57]:
print(reviews1["Package Name"].value_counts())
print(reviews2["Package Name"].value_counts())
print(reviews3["Package Name"].value_counts())
print(reviews4["Package Name"].value_counts())
print(reviews5["Package Name"].value_counts())

com.google.android.diskusage             1000
com.google.android.apps.nbu.files        1000
com.mobile_infographics_tools.mydrive    1000
com.avast.android.cleaner                1000
com.mobisystems.fileman                  1000
Name: Package Name, dtype: int64
com.financial.calculator                       1000
advanced.scientific.calculator.calc991.plus    1000
com.rkayapps.compoundinterestcalculator         854
ee.smkv.calc.loan                               549
com.financial.loan.emi.calculator               220
Name: Package Name, dtype: int64
com.han.dotsandboxes                1000
com.outofthebit.dots                 871
com.mggames.dotsnbox                 431
de.stefan_oltmann.kaesekaestchen       8
Name: Package Name, dtype: int64
com.google.android.calendar        1000
com.itbenefit.android.calendar     1000
com.moonly.android                  711
com.bihsnow.monthcalendarwidget     617
Name: Package Name, dtype: int64
com.burakgon.dnschanger            1000
free.vpn.unblo

**In the Assigned APPS, the "DiskUsage" with package name 'com.google.android.diskusage' has 1000 lines.**

**II.From its reviews, extract minimum of 10 LDA topics each being described by at least 7 Words.**

In [58]:
LDA_app = 'com.google.android.diskusage'
LDA_reviews = Vader_polarity[Vader_polarity["App's Package Name"] == LDA_app]


In [59]:
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from pprint import pprint
import gensim.corpora as corpora
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = LDA_reviews.Review.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['absolutely', 'love', 'app', 'design', 'easy']


In [60]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus_diskusage = [id2word.doc2bow(text) for text in texts]
# View
print(corpus_diskusage[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


**The table for the topics**

In [61]:
# number of topics
num_topics = 10
# Build LDA model
lda_model_diskusage = gensim.models.LdaMulticore(corpus=corpus_diskusage,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model_diskusage.print_topics())
doc_lda = lda_model_diskusage[corpus_diskusage]

[(0,
  '0.057*"app" + 0.023*"file" + 0.019*"storage" + 0.014*"space" + 0.013*"show" '
  '+ 0.012*"great" + 0.012*"simple" + 0.010*"used" + 0.010*"delete" + '
  '0.010*"good"'),
 (1,
  '0.032*"app" + 0.021*"best" + 0.018*"one" + 0.017*"android" + 0.012*"space" '
  '+ 0.012*"super" + 0.012*"easy" + 0.010*"delete" + 0.009*"apps" + '
  '0.009*"clear"'),
 (2,
  '0.061*"app" + 0.019*"android" + 0.016*"space" + 0.013*"storage" + '
  '0.011*"best" + 0.010*"exactly" + 0.009*"super" + 0.008*"phone" + '
  '0.007*"dont" + 0.007*"thank"'),
 (3,
  '0.032*"useful" + 0.030*"app" + 0.023*"best" + 0.016*"file" + 0.016*"simple" '
  '+ 0.015*"easy" + 0.015*"storage" + 0.014*"phone" + 0.013*"space" + '
  '0.013*"apps"'),
 (4,
  '0.049*"app" + 0.021*"useful" + 0.021*"storage" + 0.020*"simple" + '
  '0.016*"space" + 0.016*"work" + 0.014*"file" + 0.013*"phone" + 0.010*"see" + '
  '0.010*"android"'),
 (5,
  '0.048*"app" + 0.029*"storage" + 0.026*"good" + 0.013*"apps" + 0.013*"file" '
  '+ 0.012*"show" + 0.010*

**III. Discussion:**


In [62]:
# display the manual features
LDA_app_features = features[features['package name'] == LDA_app][['feature']]
LDA_app_features

,feature
0,Visually explore used and free space on intern...
1,find files and directories on storage card whi...
2,displays a diagram with directories subdirecto...
3,OI FileManager
4,Solid Explorer


In [63]:
# display the bigrams of the app
LDA_app_bigrams = bigrams[bigrams['package name'] == LDA_app]['2 grams']
print(LDA_app_bigrams)

0                       visual explor
1                          explor use
2                            use free
3                          free space
4                        space intern
5                       intern extern
6                       extern storag
7                     storag diskusag
8                     diskusag provid
9                          provid way
10                           way find
11                          find file
12                     file directori
13                   directori storag
14                        storag card
15                        card consum
16                         consum lot
17                          lot space
18                      space display
19                    display diagram
20    diagram directoriessubdirectori
21      directoriessubdirectori shown
22                      shown proport
23                       proport size
24                         size press
25                         press menu
26          

In [64]:
# display the trigrams for the app
LDA_app_trigrams = trigrams[trigrams['package name'] == LDA_app]['3 grams']
print(LDA_app_trigrams)

0                           visual explor use
1                             explor use free
2                              use free space
3                           free space intern
4                         space intern extern
5                        intern extern storag
6                      extern storag diskusag
7                      storag diskusag provid
8                         diskusag provid way
9                             provid way find
10                              way find file
11                        find file directori
12                      file directori storag
13                      directori storag card
14                         storag card consum
15                            card consum lot
16                           consum lot space
17                          lot space display
18                      space display diagram
19    display diagram directoriessubdirectori
20      diagram directoriessubdirectori shown
21      directoriessubdirectori sh

***Discussion:*** <br>
**After comparing the topics, features, bigrams and trigrams. We can see the most topics are related and have overlapping with some features. At the same time, the word in topics have a good hit rate in the bigrams and trigrams.** <br>
**For example, the first topic has "app","space","like","file","interface","intuitive","show","easy","storage","know". This capture the first feature "Visually explore used and free space on internal and external storage" with some performance description.**

**IV extract topics of reviews for competitor and similar apps**

In [65]:
# define the function for different app
def extract_topic(LDAreviews):
  exdata = LDAreviews.Review.values.tolist()
  exdata_words = list(sent_to_words(exdata))
  # remove stop words
  exdata_words = remove_stopwords(exdata_words)
  # Create Dictionary
  exid2word = corpora.Dictionary(exdata_words)
  # Create Corpus
  extexts = exdata_words
  # Term Document Frequency
  excorpus = [exid2word.doc2bow(text) for text in extexts]
  num_topics = 10
  # Build LDA model
  lda_model = gensim.models.LdaMulticore(corpus=excorpus, id2word=exid2word, num_topics=num_topics)
  # Print the Keyword in the 10 topics
  pprint(lda_model.print_topics())
  return excorpus, lda_model

**The first competitor: Files by Google**
<br> 'com.google.android.apps.nbu.files'

In [66]:
LDA_app_competitor1 = lists['package name'].iloc[1]
print(LDA_app_competitor1)
LDA_app_competitor_reviews1 = Vader_polarity[Vader_polarity["App's Package Name"] == LDA_app_competitor1]
corpus_competitor1, lda_model_competitor1 = extract_topic(LDA_app_competitor_reviews1)

com.google.android.apps.nbu.files
[(0,
  '0.057*"great" + 0.033*"app" + 0.022*"best" + 0.017*"apps" + 0.015*"nahin" + '
  '0.014*"phone" + 0.014*"tha" + 0.012*"nice" + 0.012*"helpful" + '
  '0.012*"amazing"'),
 (1,
  '0.031*"good" + 0.029*"app" + 0.025*"work" + 0.018*"easy" + 0.014*"space" + '
  '0.011*"storage" + 0.011*"way" + 0.011*"like" + 0.011*"fantastic" + '
  '0.009*"first"'),
 (2,
  '0.044*"easy" + 0.020*"file" + 0.019*"thanks" + 0.014*"bad" + 0.012*"app" + '
  '0.011*"useful" + 0.011*"awesome" + 0.008*"phone" + 0.008*"space" + '
  '0.008*"lot"'),
 (3,
  '0.046*"good" + 0.044*"file" + 0.037*"app" + 0.023*"google" + '
  '0.019*"storage" + 0.015*"like" + 0.013*"thanks" + 0.013*"system" + '
  '0.011*"best" + 0.009*"excellent"'),
 (4,
  '0.093*"nice" + 0.030*"excellent" + 0.025*"file" + 0.025*"amazing" + '
  '0.017*"app" + 0.017*"phone" + 0.017*"useful" + 0.014*"great" + '
  '0.014*"application" + 0.014*"super"'),
 (5,
  '0.082*"app" + 0.080*"good" + 0.021*"nice" + 0.018*"clean" + 

**The second competitor: Storage Analyzer & Disk Usage**
<br> 'com.mobile_infographics_tools.mydrive'

In [67]:
LDA_app_competitor2 = lists['package name'].iloc[2]
print(LDA_app_competitor2)
LDA_app_competitor_reviews2 = Vader_polarity[Vader_polarity["App's Package Name"] == LDA_app_competitor2]
corpus_competitor2, lda_model_competitor2 = extract_topic(LDA_app_competitor_reviews2)

com.mobile_infographics_tools.mydrive
[(0,
  '0.049*"app" + 0.049*"file" + 0.022*"work" + 0.019*"delete" + '
  '0.019*"storage" + 0.012*"phone" + 0.012*"space" + 0.011*"data" + '
  '0.010*"find" + 0.009*"android"'),
 (1,
  '0.033*"file" + 0.031*"nice" + 0.026*"app" + 0.022*"storage" + 0.014*"love" '
  '+ 0.012*"apps" + 0.011*"gb" + 0.011*"others" + 0.009*"show" + 0.009*"easy"'),
 (2,
  '0.112*"app" + 0.079*"good" + 0.040*"file" + 0.028*"best" + 0.020*"delete" + '
  '0.019*"storage" + 0.013*"application" + 0.010*"great" + 0.009*"excellent" + '
  '0.009*"like"'),
 (3,
  '0.045*"app" + 0.032*"best" + 0.027*"file" + 0.019*"delete" + 0.016*"love" + '
  '0.016*"storage" + 0.014*"op" + 0.012*"easy" + 0.011*"thank" + 0.008*"aap"'),
 (4,
  '0.066*"app" + 0.030*"file" + 0.021*"great" + 0.017*"phone" + 0.016*"help" + '
  '0.014*"clean" + 0.011*"gb" + 0.011*"space" + 0.010*"delete" + '
  '0.009*"please"'),
 (5,
  '0.073*"app" + 0.035*"file" + 0.023*"useful" + 0.022*"phone" + '
  '0.019*"delete" + 

**The third competitor: Avast Cleanup**
<br> 'com.avast.android.cleaner'

In [68]:
LDA_app_competitor3 = lists['package name'].iloc[3]
print(LDA_app_competitor3)
LDA_app_competitor_reviews3 = Vader_polarity[Vader_polarity["App's Package Name"] == LDA_app_competitor3]
corpus_competitor3, lda_model_competitor3 = extract_topic(LDA_app_competitor_reviews3)

com.avast.android.cleaner
[(0,
  '0.053*"good" + 0.040*"app" + 0.018*"love" + 0.015*"free" + 0.014*"ad" + '
  '0.013*"version" + 0.012*"working" + 0.011*"phone" + 0.011*"avast" + '
  '0.010*"well"'),
 (1,
  '0.075*"app" + 0.040*"good" + 0.021*"great" + 0.021*"clean" + 0.014*"nice" + '
  '0.012*"apps" + 0.011*"excellent" + 0.010*"avast" + 0.010*"like" + '
  '0.009*"cache"'),
 (2,
  '0.044*"work" + 0.034*"app" + 0.022*"great" + 0.016*"clean" + 0.016*"ad" + '
  '0.015*"best" + 0.012*"phone" + 0.011*"cleaning" + 0.011*"good" + '
  '0.011*"awesome"'),
 (3,
  '0.027*"app" + 0.020*"avast" + 0.019*"nice" + 0.016*"phone" + 0.013*"apps" + '
  '0.013*"easy" + 0.012*"work" + 0.010*"like" + 0.010*"free" + 0.009*"ad"'),
 (4,
  '0.076*"good" + 0.039*"app" + 0.036*"avast" + 0.012*"easy" + 0.012*"clean" + '
  '0.009*"using" + 0.009*"best" + 0.008*"year" + 0.007*"ever" + '
  '0.007*"really"'),
 (5,
  '0.028*"app" + 0.027*"good" + 0.025*"phone" + 0.022*"clean" + 0.014*"keep" + '
  '0.012*"great" + 0.011*

**The similar: File Commander**
<br> 'com.mobisystems.fileman'

In [69]:
LDA_app_similar = lists['package name'].iloc[4]
print(LDA_app_similar)
LDA_app_similar_reviews = Vader_polarity[Vader_polarity["App's Package Name"] == LDA_app_similar]
corpus_similar, lda_model_similar = extract_topic(LDA_app_similar_reviews)

com.mobisystems.fileman
[(0,
  '0.087*"good" + 0.049*"app" + 0.012*"ad" + 0.010*"time" + 0.009*"file" + '
  '0.009*"useful" + 0.009*"one" + 0.008*"dont" + 0.008*"would" + 0.007*"even"'),
 (1,
  '0.052*"app" + 0.028*"good" + 0.017*"nice" + 0.013*"file" + 0.012*"useful" + '
  '0.010*"excellent" + 0.009*"ad" + 0.009*"best" + 0.009*"love" + 0.008*"get"'),
 (2,
  '0.017*"ok" + 0.017*"file" + 0.013*"app" + 0.011*"money" + 0.011*"great" + '
  '0.010*"easy" + 0.009*"subscription" + 0.009*"cant" + 0.009*"even" + '
  '0.007*"many"'),
 (3,
  '0.034*"file" + 0.032*"app" + 0.029*"excellent" + 0.024*"ok" + 0.012*"nice" '
  '+ 0.012*"manager" + 0.011*"best" + 0.010*"folder" + 0.010*"one" + '
  '0.007*"thank"'),
 (4,
  '0.041*"good" + 0.036*"file" + 0.025*"best" + 0.021*"app" + 0.019*"nice" + '
  '0.010*"manager" + 0.010*"free" + 0.008*"apps" + 0.007*"work" + '
  '0.007*"phone"'),
 (5,
  '0.043*"good" + 0.026*"file" + 0.024*"work" + 0.022*"app" + 0.016*"great" + '
  '0.012*"mm" + 0.010*"still" + 0.010

**V. Comparison:**<br>
**They share many high frequence keyword. There are functionality words such as "manager", "file" and description words such as "good" and "excellent". So we can see how those apps can be recognized as either compatitor or similar apps.**
<br>
**However, they are different in some details. For example, the "File commander" app have 'pay' as keyword in its topic. But none of the others can show the same key word.**

# BONUS for TASK 4

**I. Retrieve the topic for each review of your ASSIGNED APP**

In [70]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pandas.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pandas.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pandas.Series(texts)
    sent_topics_df = pandas.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_diskusage, corpus=corpus_diskusage, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,6.0,0.8500,"app, storage, best, file, phone, space, love, ...","[absolutely, love, app, design, easy]"
1,1,8.0,0.7749,"app, great, android, simple, storage, best, wo...","[doesnt, work, anymore]"
2,2,5.0,0.5500,"app, storage, good, apps, file, show, thanks, ...",[good]
3,3,8.0,0.8199,"app, great, android, simple, storage, best, wo...","[great, app, exactly, supposed]"
4,4,6.0,0.5149,"app, storage, best, file, phone, space, love, ...","[great, app, searching, something, similar, wi..."
...,...,...,...,...,...
995,995,4.0,0.7000,"app, useful, storage, simple, space, work, fil...","[totally, work]"
996,996,7.0,0.5500,"storage, app, simple, great, apps, file, perfe...",[perfect]
997,997,1.0,0.8875,"app, best, one, android, space, super, easy, d...","[efficient, took, second, resolve, taken, day,..."
998,998,4.0,0.8875,"app, useful, storage, simple, space, work, fil...","[love, simple, show, exactly, whats, using, sp..."


**II. Provide a list of 10 Reviews that you believe developers should take care of with high priority. Discuss your logic for this selection.**

In [71]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pandas.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pandas.concat([sent_topics_sorteddf_mallet, 
                    grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], axis=0)

# Reset Index    
#sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
580,0.0,0.9827,"app, file, storage, space, show, great, simple...","[awesome, storage, analyzer, concise, precise,..."
197,1.0,0.9710,"app, best, one, android, space, super, easy, d...","[fast, ez, messy, result, hast, separate, grou..."
212,2.0,0.9719,"app, android, space, storage, best, exactly, s...","[great, managing, space, usage, old, tablet, r..."
42,3.0,0.9757,"useful, app, best, file, simple, easy, storage...","[still, working, sonezero, android, onetwo, ge..."
720,4.0,0.9830,"app, useful, storage, simple, space, work, fil...","[twofour, jan, twozero, bug, report, android, ..."
39,5.0,0.9820,"app, storage, good, apps, file, show, thanks, ...","[hat, app, app, really, generous, feature, nee..."
189,6.0,0.9842,"app, storage, best, file, phone, space, love, ...","[good, app, small, serious, problem, like, nin..."
157,7.0,0.9833,"storage, app, simple, great, apps, file, perfe...","[used, app, six, year, simply, best, edit, im,..."
501,8.0,0.9816,"app, great, android, simple, storage, best, wo...","[hasta, que, descubr, esta, aplicaci, supe, la..."
381,9.0,0.9791,"space, work, app, need, simple, phone, awesome...","[literally, best, storage, analyser, app, ive,..."


**Discussion:**<br>
**With using the 'Gensim' library, I can get the percentage matrix of the topic contribution. Reformat the matrix and take the percentage of contribution as the criteria to pick the 10 most high priority reviews. This percentege is larger if the review has more information contained.**

# **Task5**

In [72]:
#show the top 10 reviews about the 10 topics
task5 = reviews1.iloc[sent_topics_sorteddf_mallet.index]
task5

,Package Name,Reviewer name,Review,Rating
580,com.google.android.diskusage,ßÓb ßýérs,Awesome! No other storage analyzer is so conci...,5
197,com.google.android.diskusage,Sergio Dro,"Fast, ez to use, but messy the result, hast to...",3
212,com.google.android.diskusage,Pishtoff Withguglpluss,It's been great for managing my space usage on...,4
42,com.google.android.diskusage,A Google User,Still working on S10 on Android 12. You will g...,5
720,com.google.android.diskusage,A Google user,"24 Jan 20, *Bug* report: Under Android Pie (9....",4
39,com.google.android.diskusage,I'm looking for Woody.!,Hat Off to this App because this App is really...,5
189,com.google.android.diskusage,G M St-St,V good app Small but serious problem:. have li...,4
157,com.google.android.diskusage,Kevin Columbus,I have used this app for over 6 years. It is s...,5
501,com.google.android.diskusage,Cristian Brotons,"Hasta que no descubrí esta aplicación, no supe...",5
381,com.google.android.diskusage,jade.mp4,This is literally the BEST storage analyser ap...,5


**Suggestion 1: This app need the support of audioble audiobooks file type.** <br>
**Based on the review:**"V good app Small but serious problem:. have like 90g of "system data". I know what most of this is: downloaded audible audiobooks audible (large audible listening queue), and poss some downloaded podcasts. Can always clean out audible app if need space. (can't xfer books to SD card [pixel]) (audible app design flaws [re v large audible libraries] make library mgmt difficult otherwise) Since all this is clearly "media" I wish the DiskUsage app showed the breakdown into audible books etc." <br>
<br>
**Reason:** <br>
1. This review is the dominant review in task4.<br>
2. The audio is a keyword in the topics and it doesn't have any appreance in the compititor apps. So this is a advantage in the compitation.


**Suggestion 2: The app need to check the support for various Android versions.** <br>
**Based on the review:**<br>
1."Still working on S10 on Android 12. You will get a clear graphical view on your smartphone storage. Search for 'google' in the App. Or 'cache' or 'data'. It will calculate the sum over your search results and display it graphically. This is really helping you if Google Files shows a System size if 22 GB or Samsung My Files shows huge Other category 👌🙂" <br>
2."24 Jan 20, *Bug* report: Under Android Pie (9.0), opening a search now causes "Show" button to disappear! (Should be on right of the "X" used to close a search.) 14 Nov 18: Superb app. NO ads, NO outbound 'Net traffic: Simply a terrific way to see not only where and how precious storage space is being used, but also (even more importantly) to identify apps that needlessly store hundreds of MB of unnecessary, non-cache "data" – which can then be safely, manually deleted. Outstandingly useful!"<br>
<br>
**Reason:** <br>
1. They are the dominant review in task4.<br>
2. We can see that this app can work on Android 12 but not on Android Pie (9.0). So there must be some file structure or configuration issues on the app.

# BONUS for TASK 5

**Link for the first issue: <br>**
https://github.com/IvanVolosyuk/diskusage/issues/74

**Link for the second issue:**<br>
https://github.com/IvanVolosyuk/diskusage/issues/73